In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 5.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset
import sacrebleu

In [4]:
df = pd.read_csv("/kaggle/input/mansi-mono/mansi.csv", index_col=0)
df = df.rename(columns={'0': 'mns'})

In [5]:
df

,mns
0,"Татьяна Петровна Конева, а̄ги о̄парищ наме Пуз..."
1,"Оматэ Анна Никоновна, а̄ще Пётр Ильич Пузиныг ..."
2,"Татьяна иӈ ма̄ниг о̄лыс, а̄ще холас."
3,Анна Никоновна такке̄т ня̄врамыт янмалтасанэ.
4,Татьяна Саранпа̄выл школа а̄стламе юи-па̄лт 19...
...,...
203197,«Искусство Центр» ёмщакв рӯпитан ма̄гыс округ...
203199,Тот пуссын аквъёт хӯрумса̄т хус хо̄тпа рӯпиты.
203201,Акв колнакт ма̄нь пӯльница рӯпиты ос э̄лаль ...
203202,"Ня̄врамыт тотыглан ма̄гыс ма̄шинат о̄ньще̄гыт,..."


In [6]:
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, MBartForConditionalGeneration, MBart50Tokenizer
import sacrebleu
import pandas as pd
import random
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
random.seed(42)

In [9]:
device

device(type='cpu')

In [8]:
model_name = "/kaggle/input/3checkpoint"
model = M2M100ForConditionalGeneration.from_pretrained(model_name).to(device)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)

In [10]:
def translate(sentence):
    inputs = tokenizer(sentence, truncation=True, padding='max_length', max_length=128)
    
    # Move input_ids to the same device as the model
    input_ids = torch.tensor(inputs['input_ids']).unsqueeze(0).to(device)
    
    # Generate translation
    with torch.no_grad():
        generated_ids = model.generate(input_ids, max_length=200, num_beams=5, early_stopping=True)

    # Decode the generated ids
    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    pred = pred.replace('__en__ ', '')
    
    return pred

In [11]:
tqdm.pandas()

In [ ]:
df['rus_synth'] = df['mns'].progress_apply(translate)
df['mns_synth'] = df['rus_synth'].progress_apply(translate)

  0%|          | 25/183124 [01:51<223:00:16,  4.38s/it]